<a href="https://colab.research.google.com/github/matcocco/Canvas/blob/main/Missing_Assignments_Canvas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pip
!pip install canvasapi
import json
import logging
import sys
from canvasapi import Canvas # pip install canvasapi
import csv
import concurrent.futures
from functools import partial
import pandas as pd


     |████████████████████████████████| 81kB 3.6MB/s 
  Created wheel for canvasapi: filename=canvasapi-2.1.0-cp37-none-any.whl size=100262 sha256=2ac2d752095cabf7e26ee0c3b3d8d4774e341cf12aedb2561fb3bcb19f5ee423
  Stored in directory: /root/.cache/pip/wheels/e0/18/36/3025ecef40bb524bf9d6edb3a98008f5a303d9955793638d28
Successfully built canvasapi


In [ ]:
#Nth GradeAssignments 

#Insert your school name
from canvasapi import Canvas


KEY = '' # Your Canvas API key
URL = 'https://<schoolname>.instructure.com/' # Your Canvas API URL replace the school name
COURSE = '' # Your course ID Course id can be found after you enter into the course https://<schoolname>.instructure.com/courses/<ID>

canvas = Canvas(URL, KEY)
course = canvas.get_course(COURSE)
assignments = course.get_assignments()

#>>> print(courses)
#<PaginatedList of type Course>

# Access the first element in our list.
#
# You'll notice the first call takes a moment, but the next N-1
# elements (where N = the per_page argument supplied; the default is 10)
# will be instantly accessible.

# Iterate over our course list
assignmentList = []
for assignment in assignments:
    assignmentElement = {}
    assignmentElement["assignment"] = assignment.name
    assignmentElement["id"] = assignment.id
    assignmentElement["link"] = assignment.html_url
    assignmentElement["unlock_at"] = assignment.unlock_at
    assignmentElement["due_at"] = assignment.due_at
    assignmentElement["workflow_state"] = assignment.workflow_state
    assignmentElement["has_submitted_submissions"] = assignment.has_submitted_submissions

    """
    print(assignment)
    print(assignment.unlock_at)
    print(assignment.due_at)
    print(assignment.workflow_state)
    print(assignment.has_submitted_submissions)
    print(assignment.html_url)
    """
    assignmentList.append(assignmentElement)

with open('NameOfAllAssignmentsinCourseX.txt', 'w') as outfile: #Change name of file to represent the specific course
    json.dump(assignmentList, outfile)

canvas = Canvas(URL, KEY)
course = canvas.get_course(COURSE)
assignments = len(list(course.get_assignments()))
writer = csv.writer(open('CourseNameMissingAssignmentX.csv', 'w')) #Change name of file to represent the specific course

def main():
    sections = course.get_sections()

    writer.writerow(['Name', 'Subject', 'Last Activity', 'Complete', 'Missing','Links'])

    for section in sections:
        enrollments = section.get_enrollments(state="active", type="StudentEnrollment")
        
        # Play with the number of workers.
        with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
            
            data = []
            job = partial(process_user, section=section)

            results = [executor.submit(job, enrollment) for enrollment in enrollments]
        
            for f in concurrent.futures.as_completed(results):
                data.append(f.result())
                print(f'Processed {len(data)} in {len(list(enrollments))} at {section}')
                
        writer.writerows(data)

def process_user(enrollment, section):
    links = get_user_missing_Link(section, enrollment.user['id'])
    missing = get_user_missing(section, enrollment.user['id'])
    return [ 
        enrollment.user['sortable_name'], 
        section.name, 
        enrollment.last_activity_at, 
        len(missing), '; '.join(missing) , '; '.join(links)
        
    ]

def get_user_missing(section, user_id):
    submissions = section.get_multiple_submissions(student_ids=[user_id], 
                                                   include=["assignment", "assignment_id"], 
                                                   workflow_state="unsubmitted")
    missing_list = [item.assignment['name'] for item in submissions
        if item.workflow_state == "unsubmitted" and item.excused is not True]
    return missing_list


def get_user_missing_Link(section, user_id):
    submissions = section.get_multiple_submissions(student_ids=[user_id], 
                                                   include=["assignment", "assignment_id"], 
                                                   workflow_state="unsubmitted")
    missing_link = [item.assignment['html_url'] for item in submissions
        if item.workflow_state == "unsubmitted" and item.excused is not True]
    return missing_link

if __name__ == "__main__":
    main()
